In [1]:
from libs.alibaba.alibaba import Alibaba

from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import html
import time
import re

import types

In [2]:
lid = 'eyelash@glittergroupcn.com'
lpwd = 'Wangzhen68970747'
alibaba = Alibaba(lid, lpwd)

In [3]:
alibaba.login()
browser = alibaba.browser

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [32]:
def get_posted_product_info(self, pn):
    browser = self.browser
    products = []
    try:
        self.notify("primary", '打开 产品管理 页面 ... ...')
        browser.get(self.api_product_manage)
        css_selector = '#ballon-container .list-item'
        WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))

        msg = '切换 至 显示全部产品 ... ...'
        self.notify("primary", msg)
        css_selector = '#ballon-container div[role="tab"]:nth-child(1)'
        tab_all = WebDriverWait(browser, 15).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector)))
        tab_all.click()
        browser.implicitly_wait(1)
        css_selector = '#ballon-container .manage-loading .next-loading-tip'
        div_tip = browser.find_elements_by_css_selector(css_selector)
        if len(div_tip) != 0:
            WebDriverWait(browser, 20).until(EC.staleness_of(div_tip[0]))

        pn = int(pn)
        for counter in range(pn):
            msg = '查找 全部产品列表 第 ' + str(counter + 1) + ' 页'
            self.notify("primary", msg)

            css_selector = '#ballon-container .list-item'
            html_source = browser.page_source
            doc = pq(html_source)
            items = doc('#ballon-container .list-item')
            for item in items:
                pq_item = pq(item)
                product = {}
                pq_a = pq_item.find('.product-subject a')
                if len(pq_a) == 0:
                    continue
                
                product['href'] = pq_a.attr('href')
                product['title'] = pq_a.text().strip().lower()
                product['pid'] = pq_item.find('.product-model').text().split(':')[1].strip()
                product['category'] = pq_item.find('.product-group').text().split(':')[1].strip()
                product['update'] = pq_item.find('.product-update span').text().strip()
                product['price'] = pq_item.find('.product-price div').text().strip()
                product['quality'] = []
                product['quality'].append(pq_item.find('.product-quality div:nth-child(1)').text().strip())
                product['quality'].append(pq_item.find('.product-quality div:nth-child(2)').text().strip())
                pq_tags = pq_item.find('.product-tags .next-tag-body')
                product['tags'] = []
                
                for tag in pq_tags:
                    product['tags'].append(pq(tag).text().strip())
                
                result = re.search('id=(\d+)$', product['href'])
                if result:
                    ali_id = result.group(1)
                else:
                    ali_id = re.search('_(\d+).htm', product['href']).group(1)
                product['ali_id'] = ali_id
                products.append(product)

            if counter == pn - 1:
                break

            css_selector = '#ballon-container .next-pagination-pages .next'
            btn_next = browser.find_element_by_css_selector(css_selector)
            if btn_next.is_enabled():
                btn_next.click()
                browser.implicitly_wait(1)
                css_selector = '#ballon-container .manage-loading .next-loading-tip'
                div_tip = browser.find_elements_by_css_selector(css_selector)
                if len(div_tip) != 0:
                    WebDriverWait(browser, 20).until(EC.staleness_of(div_tip[0]))
            else:
                break

            self.notify("primary", "完成 查找！")
    except Exception as e:
        self.notify("danger", "爬取产品 [" + ali_id + "] 数据 出错, " + str(e))
        products = None
    finally:
        if self.socketio:
            self.socketio.emit("get_posted_product_info_result", products)
    print(products)
alibaba.get_posted_product_info = types.MethodType(get_posted_product_info, alibaba)

In [4]:
data = {'ali_id': 60763545453, 'price': {'isTieredPricing': False, 'price_range': ['3.3', '4.6'], 'tieredPricing': []}, 'detail_pictures': [{'old': '//sc02.alicdn.com/kf/HTB156nci3KTBuNkSne1q6yJoXXaO/231186930/HTB156nci3KTBuNkSne1q6yJoXXaO.jpg', 'new': '//sc02.alicdn.com/kf/HTB1cn7DpvuSBuNkHFqDq6xfhVXaT/231186930/HTB1cn7DpvuSBuNkHFqDq6xfhVXaT.jpg'}, {'old': '//sc02.alicdn.com/kf/HTB1KAzci3KTBuNkSne1q6yJoXXas/231186930/HTB1KAzci3KTBuNkSne1q6yJoXXas.jpg', 'new': '//sc01.alicdn.com/kf/HTB1sJ4Ox25TBuNjSspmq6yDRVXax/231186930/HTB1sJ4Ox25TBuNjSspmq6yDRVXax.jpg'}, {'old': '//sc01.alicdn.com/kf/HTB1xBa5iZuYBuNkSmRyq6AA3pXaY/231186930/HTB1xBa5iZuYBuNkSmRyq6AA3pXaY.jpg', 'new': '//sc02.alicdn.com/kf/HTB176d8x49YBuNjy0Ffq6xIsVXaE/231186930/HTB176d8x49YBuNjy0Ffq6xIsVXaE.jpg'}]}
data
alibaba.update_product(data)

primary 打开产品 [60763545453] 编辑页面 ... ...
{'ali_id': 60763545453, 'price': {'isTieredPricing': False, 'price_range': ['3.3', '4.6'], 'tieredPricing': []}, 'detail_pictures': [{'old': '//sc02.alicdn.com/kf/HTB156nci3KTBuNkSne1q6yJoXXaO/231186930/HTB156nci3KTBuNkSne1q6yJoXXaO.jpg', 'new': '//sc02.alicdn.com/kf/HTB1cn7DpvuSBuNkHFqDq6xfhVXaT/231186930/HTB1cn7DpvuSBuNkHFqDq6xfhVXaT.jpg'}, {'old': '//sc02.alicdn.com/kf/HTB1KAzci3KTBuNkSne1q6yJoXXas/231186930/HTB1KAzci3KTBuNkSne1q6yJoXXas.jpg', 'new': '//sc01.alicdn.com/kf/HTB1sJ4Ox25TBuNjSspmq6yDRVXax/231186930/HTB1sJ4Ox25TBuNjSspmq6yDRVXax.jpg'}, {'old': '//sc01.alicdn.com/kf/HTB1xBa5iZuYBuNkSmRyq6AA3pXaY/231186930/HTB1xBa5iZuYBuNkSmRyq6AA3pXaY.jpg', 'new': '//sc02.alicdn.com/kf/HTB176d8x49YBuNjy0Ffq6xIsVXaE/231186930/HTB176d8x49YBuNjy0Ffq6xIsVXaE.jpg'}]}
primary 设置产品 [60763545453] 的价格区间 ... ...


primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [1]:
pictures = []
picture = {'old': "//sc02.alicdn.com/kf/HTB156nci3KTBuNkSne1q6yJoXXaO/231186930/HTB156nci3KTBuNkSne1q6yJoXXaO.jpg"}
picture['new'] = "//sc02.alicdn.com/kf/HTB1cn7DpvuSBuNkHFqDq6xfhVXaT/231186930/HTB1cn7DpvuSBuNkHFqDq6xfhVXaT.jpg"
pictures.append(picture)
picture = {'old': "//sc02.alicdn.com/kf/HTB1KAzci3KTBuNkSne1q6yJoXXas/231186930/HTB1KAzci3KTBuNkSne1q6yJoXXas.jpg"}
picture['new'] = "//sc01.alicdn.com/kf/HTB1sJ4Ox25TBuNjSspmq6yDRVXax/231186930/HTB1sJ4Ox25TBuNjSspmq6yDRVXax.jpg"
pictures.append(picture)
picture = {'old': "//sc01.alicdn.com/kf/HTB1xBa5iZuYBuNkSmRyq6AA3pXaY/231186930/HTB1xBa5iZuYBuNkSmRyq6AA3pXaY.jpg"}
picture['new'] = "//sc02.alicdn.com/kf/HTB176d8x49YBuNjy0Ffq6xIsVXaE/231186930/HTB176d8x49YBuNjy0Ffq6xIsVXaE.jpg"
pictures.append(picture)
price = {'isTieredPricing': False, 'price_range': [3.3, 4.8], 'tieredPricing': []}
data = dict(ali_id="60764140968", price=price, detail_pictures=pictures)

In [94]:
alibaba.update_product(data)

primary 打开产品 [60764140968] 编辑页面 ... ...
primary 设置产品 [60764140968] 的价格区间 ... ...


In [2]:
data

{'ali_id': '60764140968',
 'price': {'isTieredPricing': False,
  'price_range': [3.3, 4.8],
  'tieredPricing': []},
 'detail_pictures': [{'old': '//sc02.alicdn.com/kf/HTB156nci3KTBuNkSne1q6yJoXXaO/231186930/HTB156nci3KTBuNkSne1q6yJoXXaO.jpg',
   'new': '//sc02.alicdn.com/kf/HTB1cn7DpvuSBuNkHFqDq6xfhVXaT/231186930/HTB1cn7DpvuSBuNkHFqDq6xfhVXaT.jpg'},
  {'old': '//sc02.alicdn.com/kf/HTB1KAzci3KTBuNkSne1q6yJoXXas/231186930/HTB1KAzci3KTBuNkSne1q6yJoXXas.jpg',
   'new': '//sc01.alicdn.com/kf/HTB1sJ4Ox25TBuNjSspmq6yDRVXax/231186930/HTB1sJ4Ox25TBuNjSspmq6yDRVXax.jpg'},
  {'old': '//sc01.alicdn.com/kf/HTB1xBa5iZuYBuNkSmRyq6AA3pXaY/231186930/HTB1xBa5iZuYBuNkSmRyq6AA3pXaY.jpg',
   'new': '//sc02.alicdn.com/kf/HTB176d8x49YBuNjy0Ffq6xIsVXaE/231186930/HTB176d8x49YBuNjy0Ffq6xIsVXaE.jpg'}]}

In [89]:
element = browser.find_element_by_css_selector("#editor-container")
ActionChains(browser).move_to_element(element).perform()
# self.notify("primary", "开始 设置 产品详情页 ... ...")
# 点击 详情编辑器 ‘源代码’ 按钮， 清空内容，再切换回 富文本 编辑状态
browser.find_element_by_css_selector('#mceu_0 button').click()
textarea = WebDriverWait(browser, 5).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, '#mceu_46 textarea')))

In [90]:
text = textarea.get_attribute('value')
soup = BeautifulSoup(text, 'html.parser')
for img in soup.find_all('img'):
    src = img['src']
    key = re.sub('//[^\.]*\.', '', src)
    
    for idx, pic in enumerate(pictures):
        if key not in pic['old']:
            continue
            
        img['src'] = pic['new']
        if img.has_attr('ori-width'):
            del img['ori-width']
        if img.has_attr('ori-height'):
            del img['ori-height']
        break


In [91]:

js_templ = "document.querySelector('{selector}').value = '{value}';"
js = js_templ.format(selector='#mceu_46 textarea', value=re.sub('\n', '\\\\n', str(soup)))
browser.find_element_by_css_selector('#mceu_46 textarea').clear()
browser.execute_script(js)
browser.find_element_by_css_selector('#mceu_0 button').click()
WebDriverWait(browser, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#mceu_46 iframe')))